<a href="https://colab.research.google.com/github/Adizcool/Room_Price_Prediction/blob/main/001D_Indi_Regression_Project_predicting__Room_Prices_for_given_Customer_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1) Install/ Import the required Python Packages/ Libraries**

In [1]:
#Import required python packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
import warnings
warnings.filterwarnings("ignore")
from sklearn import preprocessing
%matplotlib inline

In [2]:
pip install category_encoders

     |████████████████████████████████| 82 kB 247 kB/s 


**2) Mounting the Google Drive**

In [3]:
# Mount the Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**3) Read the Data file and check**

In [4]:
# Read the Diabetes Data from .csv file and check the data shape (number of Rows and Columns)
train_df = pd.read_csv('gdrive/My Drive/Datasets/Room Price Prediction/Room_Price_Training_Data.csv')
print(train_df.shape)
train_df.head()

(4201, 14)


,ID,District,Neighborhood,PropertyType,CancellationPolicy,Accommodates,RoomType,Bathrooms,Bedrooms,CleaningFee,Latitude,Longitude,ReviewRating,Price
0,5300082,Manhattan,East Village,Apartment,strict,2,Entire home/apt,1.0,0.0,75.000000,40.729555,-73.981665,86.0,160.0
1,17634206,Manhattan,Hell's Kitchen,Apartment,flexible,2,Entire home/apt,1.0,1.0,66.646525,40.766988,-73.984802,100.0,130.0
2,4025329,Manhattan,Harlem,Apartment,strict,4,Entire home/apt,1.0,0.0,69.000000,40.815859,-73.941165,91.0,129.0
3,10096773,Manhattan,Chelsea,Apartment,flexible,2,Entire home/apt,1.0,1.0,60.000000,40.745773,-74.000744,100.0,145.0
4,11616592,Manhattan,West Village,Apartment,strict,9,Entire home/apt,2.5,3.0,160.000000,40.737583,-74.005531,94.0,399.0


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4201 entries, 0 to 4200
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  4201 non-null   int64  
 1   District            4201 non-null   object 
 2   Neighborhood        4201 non-null   object 
 3   PropertyType        4201 non-null   object 
 4   CancellationPolicy  4201 non-null   object 
 5   Accommodates        4201 non-null   int64  
 6   RoomType            4201 non-null   object 
 7   Bathrooms           4201 non-null   float64
 8   Bedrooms            4201 non-null   float64
 9   CleaningFee         4201 non-null   float64
 10  Latitude            4201 non-null   float64
 11  Longitude           4201 non-null   float64
 12  ReviewRating        4201 non-null   float64
 13  Price               4201 non-null   float64
dtypes: float64(7), int64(2), object(5)
memory usage: 459.6+ KB


In [6]:
# Read the Diabetes Data from .csv file and check the data shape (number of Rows and Columns)
test_df = pd.read_csv('gdrive/My Drive/Datasets/Room Price Prediction/Room_Price_Customer_Data.csv')
print(test_df.shape)
test_df.head()

(1400, 13)


,ID,District,Neighborhood,PropertyType,CancellationPolicy,Accommodates,RoomType,Bathrooms,Bedrooms,CleaningFee,Latitude,Longitude,ReviewRating
0,8760775,Manhattan,Murray Hill,Apartment,flexible,1,Private room,1.0,1.0,63.827637,40.749400,-73.978226,95.0
1,16425551,Manhattan,Chinatown,Apartment,flexible,1,Private room,1.0,1.0,35.000000,40.716220,-73.991230,95.0
2,15043585,Manhattan,Hell's Kitchen,Apartment,flexible,2,Private room,1.0,1.0,63.827637,40.761096,-73.987602,90.0
3,16516930,Manhattan,Gramercy,Apartment,strict,6,Entire home/apt,1.0,3.0,150.000000,40.736475,-73.985136,60.0
4,1891017,Manhattan,Chelsea,Apartment,strict,2,Entire home/apt,1.0,0.0,69.000000,40.750661,-74.003774,96.0


In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  1400 non-null   int64  
 1   District            1400 non-null   object 
 2   Neighborhood        1400 non-null   object 
 3   PropertyType        1400 non-null   object 
 4   CancellationPolicy  1400 non-null   object 
 5   Accommodates        1400 non-null   int64  
 6   RoomType            1400 non-null   object 
 7   Bathrooms           1400 non-null   float64
 8   Bedrooms            1400 non-null   float64
 9   CleaningFee         1400 non-null   float64
 10  Latitude            1400 non-null   float64
 11  Longitude           1400 non-null   float64
 12  ReviewRating        1400 non-null   float64
dtypes: float64(6), int64(2), object(5)
memory usage: 142.3+ KB


In [8]:
train_df['train']=1
test_df['test'] = 0

In [9]:
combined_df  = pd.concat([train_df, test_df])
combined_df.shape

(5601, 16)

In [10]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5601 entries, 0 to 1399
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5601 non-null   int64  
 1   District            5601 non-null   object 
 2   Neighborhood        5601 non-null   object 
 3   PropertyType        5601 non-null   object 
 4   CancellationPolicy  5601 non-null   object 
 5   Accommodates        5601 non-null   int64  
 6   RoomType            5601 non-null   object 
 7   Bathrooms           5601 non-null   float64
 8   Bedrooms            5601 non-null   float64
 9   CleaningFee         5601 non-null   float64
 10  Latitude            5601 non-null   float64
 11  Longitude           5601 non-null   float64
 12  ReviewRating        5601 non-null   float64
 13  Price               4201 non-null   float64
 14  train               4201 non-null   float64
 15  test                1400 non-null   float64
dtypes: flo

##**VI) Check the unique Values of each column and observe the following and take actions as required:**
* **a) Wrong Data in the columns, if any** 
* **b) Wrong format of the data in the columns, if any**
* **c) Identify the columns which need to be categorically converted to numeric values by using Nominal method/ Ordinal Method**


###**Column-0: ID**

In [11]:
combined_df['ID'].value_counts()

4859903     1
5561046     1
16783144    1
9845454     1
15954639    1
           ..
3841113     1
6554879     1
7064935     1
15602234    1
15720448    1
Name: ID, Length: 5601, dtype: int64

**Observations:**
* a) Data in this column will not be contributing to the prediction of the Depenedent variable

**Decsion:**

**We will be dropping this column**

In [12]:
combined_df.drop(['ID'], axis = 1, inplace = True)
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5601 entries, 0 to 1399
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   District            5601 non-null   object 
 1   Neighborhood        5601 non-null   object 
 2   PropertyType        5601 non-null   object 
 3   CancellationPolicy  5601 non-null   object 
 4   Accommodates        5601 non-null   int64  
 5   RoomType            5601 non-null   object 
 6   Bathrooms           5601 non-null   float64
 7   Bedrooms            5601 non-null   float64
 8   CleaningFee         5601 non-null   float64
 9   Latitude            5601 non-null   float64
 10  Longitude           5601 non-null   float64
 11  ReviewRating        5601 non-null   float64
 12  Price               4201 non-null   float64
 13  train               4201 non-null   float64
 14  test                1400 non-null   float64
dtypes: float64(9), int64(1), object(5)
memory usage: 700.1+

###**Column-1: District**

In [13]:
combined_df['District'].value_counts()

Manhattan    5504
Brooklyn       59
Queens         36
Bronx           2
Name: District, dtype: int64

**Observations:**
* a) Data in this column is of "Object" or "String" datatype. Also, the data levels are "Nominal" Type.

**Decsion:**

**We will be converting the data in this column into Numerical values using Nominal Type method "pd.get_dummies".**

**Action:**

In [14]:
dist = pd.DataFrame(combined_df['District'])
dist_encoded = pd.get_dummies(data = dist, drop_first= True)
dist_encoded

,District_Brooklyn,District_Manhattan,District_Queens
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
1395,0,1,0
1396,0,1,0
1397,0,1,0
1398,0,1,0


###**Column-2: Neighborhood**

In [15]:
combined_df['Neighborhood'].value_counts()

Harlem             733
East Village       550
Upper West Side    495
Hell's Kitchen     470
Upper East Side    465
                  ... 
Midwood              1
Clinton Hill         1
Morrisania           1
Jackson Heights      1
Maspeth              1
Name: Neighborhood, Length: 65, dtype: int64

**Observations:**
* a) Data in this column is of "Object" or "String" datatype. Also, the data levels are "Nominal" Type.

**Decsion:**

**We will be converting the data in this column into Numerical values using Nominal Type method "Binary Encoding" for minimum number of colunms**

In [16]:
#create object for binary encoding
import category_encoders as ce
encoder2 = ce.BinaryEncoder(cols = ['Neighborhood'], return_df = True)

In [17]:
#fit and transform data
neighborhood_encoded = encoder2.fit_transform(combined_df['Neighborhood'])
neighborhood_encoded

,Neighborhood_0,Neighborhood_1,Neighborhood_2,Neighborhood_3,Neighborhood_4,Neighborhood_5,Neighborhood_6
0,0,0,0,0,0,0,1
1,0,0,0,0,0,1,0
2,0,0,0,0,0,1,1
3,0,0,0,0,1,0,0
4,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...
1395,0,0,0,0,0,0,1
1396,0,0,1,0,0,1,1
1397,0,0,0,0,0,1,1
1398,0,0,0,1,1,0,1


###**Column-3: Property Type**

In [18]:
combined_df['PropertyType'].value_counts()

Apartment          5285
House                81
Loft                 71
Townhouse            53
Condominium          48
Other                28
Bed & Breakfast      13
Timeshare             8
Hostel                4
Dorm                  2
Guesthouse            2
Vacation home         1
Bungalow              1
Boutique hotel        1
Lighthouse            1
Villa                 1
Hut                   1
Name: PropertyType, dtype: int64

**Observations:**
* a) Data in this column is of "Object" or "String" datatype. Also, the data levels are "Nominal" Type.

**Decsion:**

**We will be converting the data in this column into Numerical values using Nominal Type method "Binary Encoding" for minimum number of colunms**

In [19]:
#create object for binary encoding
import category_encoders as ce
encoder2 = ce.BinaryEncoder(cols = ['PropertyType'], return_df = True)

In [20]:
#fit and transform data
propertytype_encoded = encoder2.fit_transform(combined_df['PropertyType'])
propertytype_encoded

,PropertyType_0,PropertyType_1,PropertyType_2,PropertyType_3,PropertyType_4
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1
...,...,...,...,...,...
1395,0,0,0,0,1
1396,0,0,0,0,1
1397,0,0,0,0,1
1398,0,0,0,0,1


###**Column-4: Cancellation Policy**

In [21]:
combined_df['CancellationPolicy'].value_counts()

strict      3025
moderate    1352
flexible    1224
Name: CancellationPolicy, dtype: int64

**Observations:**
* a) Data in this column is of "Object" or "String" datatype. Also, the data levels are "Ordinal" Type.

**Decsion:**

**We will be converting the data in this column into Numerical values using Ordnial Type method "preprocessing.LabelEncoder()**

In [22]:
le = preprocessing.LabelEncoder()
combined_df['CancellationPolicy'] = le.fit_transform(combined_df.CancellationPolicy.values)
combined_df['CancellationPolicy'].value_counts()

2    3025
1    1352
0    1224
Name: CancellationPolicy, dtype: int64

###**Column-5: Accommodates**

In [23]:
combined_df['Accommodates'].value_counts()

2     2747
4      863
3      701
1      613
6      288
5      205
8       85
7       52
10      23
9       11
12       6
14       3
16       2
15       1
13       1
Name: Accommodates, dtype: int64

**Observations:**
* a) Data in this column is of "int" datatype. Also, there seems to be no wrong data or wrong data format.

**Decsion:**

**Let us leave the column as it is.**

###**Column-6: Room Type**

In [24]:
combined_df['RoomType'].value_counts()

Entire home/apt    3213
Private room       2236
Shared room         152
Name: RoomType, dtype: int64

**Observations:**
* a) Data in this column is of "Object" or "String" datatype. Also, the data levels are "Ordinal" Type.

**Decsion:**

**We will be converting the data in this column into Numerical values using Ordnial Type method "preprocessing.LabelEncoder()**

In [25]:
le = preprocessing.LabelEncoder()
combined_df['RoomType'] = le.fit_transform(combined_df.RoomType.values)
combined_df['RoomType'].value_counts()

0    3213
1    2236
2     152
Name: RoomType, dtype: int64

###**Column-7: Bathrooms**

In [26]:
combined_df['Bathrooms'].value_counts()

1.000000    4998
2.000000     311
1.500000     184
2.500000      25
1.087878      24
0.000000      20
3.000000      13
0.500000      11
3.500000       7
1.061030       6
4.000000       1
5.000000       1
Name: Bathrooms, dtype: int64

**Observations:**
* a) Data in this column is of "float" datatype. Also, there seems to be no wrong data or wrong data format.

**Decsion:**

**Let us leave the column as it is.**

###**Column-8: Bedrooms**

In [27]:
combined_df['Bedrooms'].value_counts()

1.000000    4026
0.000000     698
2.000000     660
3.000000     172
4.000000      33
1.085586       5
5.000000       5
1.030153       2
Name: Bedrooms, dtype: int64

**Observations:**
* a) Data in this column is of "float" datatype. Also, there seems to be no wrong data or wrong data format.

**Decsion:**

**Let us leave the column as it is.**

###**Column-9: Cleaning Fee**

In [28]:
combined_df['CleaningFee'].value_counts()

66.646525     1029
50.000000      584
100.000000     434
63.827637      300
30.000000      257
              ... 
9.000000         1
400.000000       1
68.000000        1
0.000000         1
295.000000       1
Name: CleaningFee, Length: 121, dtype: int64

**Observations:**
* a) Data in this column is of "float" datatype. Also, there seems to be no wrong data or wrong data format.

**Decsion:**

**Let us leave the column as it is.**

###**Column-10: Latitude**

In [29]:
combined_df['Latitude'].value_counts()

40.776014    1
40.805701    1
40.767692    1
40.718527    1
40.714022    1
            ..
40.795268    1
40.721004    1
40.718924    1
40.808054    1
40.725199    1
Name: Latitude, Length: 5601, dtype: int64

**Observations:**
* a) Data in this column will not be contributing to the prediction of the Depenedent variable

**Decsion:**

**We will be dropping this column**

In [30]:
#dropping the column
combined_df.drop(columns = ['Latitude'], axis = 1, inplace = True)
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5601 entries, 0 to 1399
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   District            5601 non-null   object 
 1   Neighborhood        5601 non-null   object 
 2   PropertyType        5601 non-null   object 
 3   CancellationPolicy  5601 non-null   int64  
 4   Accommodates        5601 non-null   int64  
 5   RoomType            5601 non-null   int64  
 6   Bathrooms           5601 non-null   float64
 7   Bedrooms            5601 non-null   float64
 8   CleaningFee         5601 non-null   float64
 9   Longitude           5601 non-null   float64
 10  ReviewRating        5601 non-null   float64
 11  Price               4201 non-null   float64
 12  train               4201 non-null   float64
 13  test                1400 non-null   float64
dtypes: float64(8), int64(3), object(3)
memory usage: 656.4+ KB


###**Column-11: Longitude**

In [31]:
combined_df['Longitude'].value_counts()

-73.977991    1
-74.000565    1
-73.980365    1
-73.958880    1
-73.946893    1
             ..
-74.002958    1
-74.004247    1
-73.943143    1
-73.952518    1
-73.993500    1
Name: Longitude, Length: 5601, dtype: int64

**Observations:**
* a) Data in this column will not be contributing to the prediction of the Depenedent variable

**Decsion:**

**We will be dropping this column**

In [32]:
#dropping the column
combined_df.drop(columns = ['Longitude'], axis = 1, inplace = True)
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5601 entries, 0 to 1399
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   District            5601 non-null   object 
 1   Neighborhood        5601 non-null   object 
 2   PropertyType        5601 non-null   object 
 3   CancellationPolicy  5601 non-null   int64  
 4   Accommodates        5601 non-null   int64  
 5   RoomType            5601 non-null   int64  
 6   Bathrooms           5601 non-null   float64
 7   Bedrooms            5601 non-null   float64
 8   CleaningFee         5601 non-null   float64
 9   ReviewRating        5601 non-null   float64
 10  Price               4201 non-null   float64
 11  train               4201 non-null   float64
 12  test                1400 non-null   float64
dtypes: float64(7), int64(3), object(3)
memory usage: 612.6+ KB


###**Column-12: Review Rating**

In [33]:
combined_df['ReviewRating'].value_counts()

100.0    1464
93.0      427
95.0      402
90.0      358
96.0      342
97.0      294
80.0      286
98.0      275
94.0      248
92.0      229
91.0      177
87.0      150
99.0      144
88.0      132
89.0      132
85.0       84
84.0       62
83.0       61
60.0       60
86.0       49
70.0       33
82.0       26
78.0       20
76.0       18
75.0       16
73.0       16
81.0       15
40.0       13
67.0       12
20.0       11
77.0       10
74.0        9
79.0        7
68.0        5
72.0        4
66.0        2
65.0        2
50.0        2
53.0        2
27.0        1
47.0        1
Name: ReviewRating, dtype: int64

**Observations:**
* a) Data in this column is of "float" datatype. Also, there seems to be no wrong data or wrong data format.

**Decsion:**

**Let us leave the column as it is.**

###**Column-13: Price**

In [34]:
combined_df['Price'].value_counts()

100.0    200
150.0    164
200.0    145
250.0    124
120.0    113
        ... 
287.0      1
436.0      1
980.0      1
194.0      1
413.0      1
Name: Price, Length: 320, dtype: int64

**Observations:**
* a) Data in this column is of "float" datatype. Also, there seems to be no wrong data or wrong data format.

**Decsion:**

**Let us leave the column as it is.**

In [35]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5601 entries, 0 to 1399
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   District            5601 non-null   object 
 1   Neighborhood        5601 non-null   object 
 2   PropertyType        5601 non-null   object 
 3   CancellationPolicy  5601 non-null   int64  
 4   Accommodates        5601 non-null   int64  
 5   RoomType            5601 non-null   int64  
 6   Bathrooms           5601 non-null   float64
 7   Bedrooms            5601 non-null   float64
 8   CleaningFee         5601 non-null   float64
 9   ReviewRating        5601 non-null   float64
 10  Price               4201 non-null   float64
 11  train               4201 non-null   float64
 12  test                1400 non-null   float64
dtypes: float64(7), int64(3), object(3)
memory usage: 612.6+ KB


In [36]:
combined_df.drop(['District', 'Neighborhood', 'PropertyType'], axis = 1, inplace = True)
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5601 entries, 0 to 1399
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CancellationPolicy  5601 non-null   int64  
 1   Accommodates        5601 non-null   int64  
 2   RoomType            5601 non-null   int64  
 3   Bathrooms           5601 non-null   float64
 4   Bedrooms            5601 non-null   float64
 5   CleaningFee         5601 non-null   float64
 6   ReviewRating        5601 non-null   float64
 7   Price               4201 non-null   float64
 8   train               4201 non-null   float64
 9   test                1400 non-null   float64
dtypes: float64(7), int64(3)
memory usage: 481.3 KB


In [37]:
combined_df = pd.concat([combined_df,dist_encoded, neighborhood_encoded, propertytype_encoded], axis = 1)
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5601 entries, 0 to 1399
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CancellationPolicy  5601 non-null   int64  
 1   Accommodates        5601 non-null   int64  
 2   RoomType            5601 non-null   int64  
 3   Bathrooms           5601 non-null   float64
 4   Bedrooms            5601 non-null   float64
 5   CleaningFee         5601 non-null   float64
 6   ReviewRating        5601 non-null   float64
 7   Price               4201 non-null   float64
 8   train               4201 non-null   float64
 9   test                1400 non-null   float64
 10  District_Brooklyn   5601 non-null   uint8  
 11  District_Manhattan  5601 non-null   uint8  
 12  District_Queens     5601 non-null   uint8  
 13  Neighborhood_0      5601 non-null   int64  
 14  Neighborhood_1      5601 non-null   int64  
 15  Neighborhood_2      5601 non-null   int64  
 16  Neighb

In [38]:
train_df1 = combined_df[combined_df["train"] == 1]
test_df1 = combined_df[combined_df["test"] == 0]
train_df1.drop(["train", "test"], axis=1, inplace=True)
test_df1.drop(["test", "train", "Price"], axis=1, inplace=True)

In [39]:
train_df1.shape

(4201, 23)

In [40]:
#train_df1['Price'] = train_df1['Price'].astype(int)
train_df1['Price'].value_counts() 

100.0    200
150.0    164
200.0    145
250.0    124
120.0    113
        ... 
287.0      1
436.0      1
980.0      1
194.0      1
413.0      1
Name: Price, Length: 320, dtype: int64

In [41]:
test_df1.shape

(1400, 22)

In [42]:
X = train_df1.drop(['Price'], axis = 1)
y = train_df1['Price']

###**Ridge Regression**

In [43]:
alphas = [0.001, 0.01, 0.1, 1, 10]
print('All errors are RMSE')
print('-'*76)
for alpha in alphas:
    # instantiate and fit model
    ridge = Ridge(alpha=alpha, fit_intercept=True, random_state=99)
    ridge.fit(X, y)
    # calculate errors
    new_train_error1 = np.sqrt(mean_squared_error(y, ridge.predict(X)))
    print('alpha: {:7} | train error: {:5}'.format(alpha,round(new_train_error1,3)))

All errors are RMSE
----------------------------------------------------------------------------
alpha:   0.001 | train error:  72.5
alpha:    0.01 | train error:  72.5
alpha:     0.1 | train error:  72.5
alpha:       1 | train error: 72.507
alpha:      10 | train error: 72.551


In [44]:
ridge = Ridge(alpha=10)
ridge.fit(X, y) 
pred_train_ridge= ridge.predict(X)
Ridge_Train_Acc = r2_score(y, pred_train_ridge)
Ridge_Train_Err = np.sqrt(mean_squared_error(y,pred_train_ridge))
print("Train Error : ", np.sqrt(mean_squared_error(y,pred_train_ridge)))
print("Train Accuracy : ", r2_score(y, pred_train_ridge))

Train Error :  72.55130190106253
Train Accuracy :  0.5677528865165724


In [45]:
test_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1400 entries, 0 to 1399
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CancellationPolicy  1400 non-null   int64  
 1   Accommodates        1400 non-null   int64  
 2   RoomType            1400 non-null   int64  
 3   Bathrooms           1400 non-null   float64
 4   Bedrooms            1400 non-null   float64
 5   CleaningFee         1400 non-null   float64
 6   ReviewRating        1400 non-null   float64
 7   District_Brooklyn   1400 non-null   uint8  
 8   District_Manhattan  1400 non-null   uint8  
 9   District_Queens     1400 non-null   uint8  
 10  Neighborhood_0      1400 non-null   int64  
 11  Neighborhood_1      1400 non-null   int64  
 12  Neighborhood_2      1400 non-null   int64  
 13  Neighborhood_3      1400 non-null   int64  
 14  Neighborhood_4      1400 non-null   int64  
 15  Neighborhood_5      1400 non-null   int64  
 16  Neighb

In [46]:
pred_test_ridge = ridge.predict(test_df1)
pred_test_ridge

array([123.53792374, 100.88725681, 115.83801554, ..., 365.6519597 ,
       116.94665575, 106.38666962])

In [47]:
test_df["Predicted_Price"] = pred_test_ridge
print(test_df.shape)
test_df.head()

(1400, 15)


,ID,District,Neighborhood,PropertyType,CancellationPolicy,Accommodates,RoomType,Bathrooms,Bedrooms,CleaningFee,Latitude,Longitude,ReviewRating,test,Predicted_Price
0,8760775,Manhattan,Murray Hill,Apartment,flexible,1,Private room,1.0,1.0,63.827637,40.749400,-73.978226,95.0,0,123.537924
1,16425551,Manhattan,Chinatown,Apartment,flexible,1,Private room,1.0,1.0,35.000000,40.716220,-73.991230,95.0,0,100.887257
2,15043585,Manhattan,Hell's Kitchen,Apartment,flexible,2,Private room,1.0,1.0,63.827637,40.761096,-73.987602,90.0,0,115.838016
3,16516930,Manhattan,Gramercy,Apartment,strict,6,Entire home/apt,1.0,3.0,150.000000,40.736475,-73.985136,60.0,0,306.340653
4,1891017,Manhattan,Chelsea,Apartment,strict,2,Entire home/apt,1.0,0.0,69.000000,40.750661,-74.003774,96.0,0,178.368575


In [48]:
finalDF= test_df
finalDF['Predicted_Price'] = finalDF['Predicted_Price'].astype(int)
print(finalDF.shape)
finalDF.head()

(1400, 15)


,ID,District,Neighborhood,PropertyType,CancellationPolicy,Accommodates,RoomType,Bathrooms,Bedrooms,CleaningFee,Latitude,Longitude,ReviewRating,test,Predicted_Price
0,8760775,Manhattan,Murray Hill,Apartment,flexible,1,Private room,1.0,1.0,63.827637,40.749400,-73.978226,95.0,0,123
1,16425551,Manhattan,Chinatown,Apartment,flexible,1,Private room,1.0,1.0,35.000000,40.716220,-73.991230,95.0,0,100
2,15043585,Manhattan,Hell's Kitchen,Apartment,flexible,2,Private room,1.0,1.0,63.827637,40.761096,-73.987602,90.0,0,115
3,16516930,Manhattan,Gramercy,Apartment,strict,6,Entire home/apt,1.0,3.0,150.000000,40.736475,-73.985136,60.0,0,306
4,1891017,Manhattan,Chelsea,Apartment,strict,2,Entire home/apt,1.0,0.0,69.000000,40.750661,-74.003774,96.0,0,178


In [49]:
from google.colab import files
finalDF.to_csv("gdrive/My Drive/Datasets/Room Price Prediction/Cust_data_with_Pred_Price.csv", index = False)